# Separating participants into groups

Simple script that builds a sheet in an excel file.

## Usage

In a camp participant's spreadsheet, this script takes the paying and confirmed participants and separates them into groups by age.

Then, read the files in the `source` folder — the script gets the first `xlsx` it finds.

In [1]:
input("Press any key to start...")
print("Starting script...")

Starting script...


In [2]:
from pathlib import Path

file_folder = Path("./source")


def find_first_xlsx():
    for filename in file_folder.glob("*.xlsx"):
        if "xlsx" in str(filename):
            return Path(filename)

Turns the sheet into a dataframe and sets the phone as string (to avoid confusion with numbers and zeroes)

In [3]:
import pandas as pd

path = find_first_xlsx()
print(f"\tFound file {path}")

df = pd.read_excel(
    path,
    sheet_name="LISTA DE PAGANTES (FINANCEIRA)",
    header=9,
)

	Found file source/Acampa  2025 - VOCARE (CENTRAL).xlsx


Turns `Desistente` and `Pago` into booleans

In [4]:
print("\tFiltering out dropouts, non-paid, and team members...")

df["Desistente"] = df["Desistente"] != "NÃO"
df["Pago?"] = df["Pago?"] == "PAGANTE"

	Filtering out dropouts, non-paid, and team members...


Filters out team members — leaving only participants

In [5]:
df = df[df["Será Acampante ou Equipe?"] == "Sou participante (acampante)"]

Filters out non-paying and dropouts

In [6]:
df = df[df["Pago?"]]
df = df[~df["Desistente"]]

Remove useless columns

In [7]:
print("\tRemoving unused columns...")

df.drop(
    columns=[
        'Telefone para contato (Ex: 11XXXXXXXXX sem traço "-" e sem pontos ".")',
        "Pago?",
        "Desistente",
    ],
    inplace=True,
)

	Removing unused columns...


Order by age and gender

In [8]:
df.sort_values(["Idade"], ascending=[True], inplace=True)

Create new column and add group using **round-robyn**

In [9]:
print("\tSeparating into 6 groups...")

df["Grupo"] = 0

for i, index in enumerate(df.index):
    df.loc[index, "Grupo"] = (i % 6) + 1

	Separating into 6 groups...


Prepare new filename

In [10]:
import re

print("\tWriting xlsx file...")

pattern = r"(?<=/)(.*?)(?=\.xlsx)"
match = re.search(pattern, str(path))

if not match:
    raise Exception('Wait, is there an xlsx file inside "source" folder?')

old_name = match.group(0)
new_path = re.sub(pattern, f"{old_name} - COM GRUPOS", str(path))

	Writing xlsx file...


Write new excel file

In [11]:
import openpyxl

workbook = openpyxl.load_workbook(path)

Add a sheet with every person and their groups

In [12]:
print("\tCreating unified sheet...")

# FIRST: a unified sheet
unified_sheet_name = "Grupos (todos)"
if unified_sheet_name in workbook.sheetnames:
    unified_sheet = workbook[unified_sheet_name]
else:
    unified_sheet = workbook.create_sheet(unified_sheet_name)

# Clear sheet
unified_sheet.delete_rows(1, unified_sheet.max_row)

row_index = 1

# Write the headers
for c, col_name in enumerate(df.columns):
    cell = unified_sheet.cell(row=1, column=c + 1)
    cell.value = col_name

# Write rows
for r in range(df.shape[0]):
    for c in range(df.shape[1]):
        cell = unified_sheet.cell(row=r + 2, column=c + 1)
        cell.value = df.iloc[r, c]

# Apply AutoFilter
start_column = 1  # Column A (1)
end_column = len(df.columns)  # Number of columns
end_row = df.shape[0] + 1  # Last row including header

filter_range = f"A1:{chr(64 + end_column)}{end_row}"
unified_sheet.auto_filter.ref = filter_range

	Creating unified sheet...


Generate a sheet with separated groups

In [13]:
print("\tCreating separated sheet...")

# THEN, A SEPARATED SHEET
separated_sheet_name = "Grupos (separado)"

if separated_sheet_name in workbook.sheetnames:
    separated_sheet = workbook[separated_sheet_name]
else:
    separated_sheet = workbook.create_sheet(separated_sheet_name)

# Clear sheet
separated_sheet.delete_rows(1, separated_sheet.max_row)

dfs = {group: group_df for group, group_df in df.groupby("Grupo")}

row_index = 1

for group, group_df in dfs.items():
    separated_sheet.cell(row=row_index, column=1, value=f"Grupo {group}")
    row_index += 2  # Move down 2 rows, one for empty space

    # Write headers
    for c, col_name in enumerate(group_df.columns):
        cell = separated_sheet.cell(row=row_index, column=c + 1)
        cell.value = col_name

    row_index += 1  # Move to the next row for data

    # Write data
    for r in range(group_df.shape[0]):
        for c in range(group_df.shape[1]):
            cell = separated_sheet.cell(row=row_index + r, column=c + 1)
            cell.value = group_df.iloc[r, c]

    row_index += group_df.shape[0] + 2  # Move down after each group


	Creating separated sheet...


Save the new workbook!!

In [14]:
workbook.save(new_path)
print(f"{new_path} file written successfully.")
input("Press any key to end...")

source/Acampa  2025 - VOCARE (CENTRAL) - COM GRUPOS.xlsx file written successfully.


''